## Description du Notebook
Ce notebook effectue un traitement de texte complet (nettoyage, vectorisation TF-IDF, réduction de dimension) suivi de l'application de plusieurs modèles de machine learning :

1. **Random Forest** : Prédictions avec optimisation.
2. **SVM** (Support Vector Machine) : Optimisation avec GridSearchCV.
3. **KNN** (K-Nearest Neighbors) : Optimisation des hyperparamètres.
4. **Logistic Regression** : Modèle de base et optimisé.
5. **Decision Tree** : Analyse et évaluation.

Chaque section inclut :
- **Préparation des données** (nettoyage, TF-IDF, réduction de dimension).
- **Optimisation des hyperparamètres** (GridSearchCV).
- **Évaluation des performances** (à l'aide d'Accuracy, F1-Score, et Matrice de Confusion).
- **Graphiques explicatifs** pour visualiser les résultats.

## **Imports et Configuration**

In [ ]:
# Import des bibliothèques nécessaires
import pandas as pd
import numpy as np
import re
import spacy
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# Configurer l'affichage des graphiques
plt.style.use('ggplot')
sns.set_theme(style="whitegrid")

# Téléchargement des ressources linguistiques
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Charger le modèle SpaCy pour le français
!python -m spacy download fr_core_news_sm
nlp = spacy.load("fr_core_news_sm")
stop_words = set(stopwords.words('french'))

## **1. Prétraitement des Textes**


**Nettoyage et Lemmatisation des Textes**

In [ ]:
# Charger les données d'entrainement
data = pd.read_json("train.jsonl", lines=True)

# Fonction pour nettoyer le texte
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Fonction pour la lemmatisation
def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if token.text not in stop_words and not token.is_punct])

# Nettoyage et Lemmatisation
data['cleaned_text'] = data['texte_annonce'].apply(clean_text).apply(lemmatize_text)
print(data[['cleaned_text']].head())

**Visualisation de la longueur des textes**

In [ ]:
# Longueur des textes
text_lengths = data['cleaned_text'].apply(lambda x: len(x.split()))
plt.figure(figsize=(10, 6))
sns.histplot(text_lengths, bins=50, kde=True)
plt.title("Distribution de la longueur des textes nettoyés")
plt.xlabel("Nombre de mots")
plt.ylabel("Nombre de textes")
plt.show()

## **2. Vectorisation et Réduction de Dimension**

**TF-IDF + Truncated SVD**


In [ ]:
# Vectorisation TF-IDF
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=5, max_df=0.8)
X_tfidf = tfidf_vectorizer.fit_transform(data['cleaned_text'])

# Réduction de dimension avec SVD
svd = TruncatedSVD(n_components=100)
X_reduced = svd.fit_transform(X_tfidf)

# Variables additionnelles
data['text_length'] = data['cleaned_text'].apply(lambda x: len(x.split()))
data['contains_url'] = data['texte_annonce'].apply(lambda x: 1 if 'http' in x else 0)
data['contains_numbers'] = data['texte_annonce'].apply(lambda x: 1 if re.search(r'\d', x) else 0)

# Fusionner les variables
top_features = ['text_length', 'contains_url', 'contains_numbers']
additional_features = data[top_features].values
X_final = np.hstack((X_reduced, additional_features))

print(f"Dimensions de X_final : {X_final.shape}")

In [ ]:
# Exporter le DataFrame prétraité en CSV
# Adding the escapechar parameter to handle special characters
data.to_csv("data_preprocessed.csv", index=False, escapechar='\\')

# Ou, pour un fichier binaire plus rapide à charger
data.to_pickle("data_preprocessed.pkl")

In [ ]:
# Afficher le DataFrame transformé
print(data.head())

   OGC_FID boamp_id_annonce boamp_parent_annonce  \
0     9723        23-177335                 None   
1    10502         24-63002                 None   
2    10900         24-94957         ['24-79402']   
3      779        23-179664                 None   
4     5614         24-69326                 None   

                                   boamp_theme_boamp boamp_libelle_annonce  \
0                                      Espaces verts        Avis de marché   
1                      Station d'épuration (travaux)        Avis de marché   
2                           Voirie et réseaux divers          Rectificatif   
3  Prestations de services, Délégation de service...        Avis de marché   
4                                        Génie civil        Avis de marché   

  boamp_statut_annonce boamp_date_fin_de_marche  \
0              INITIAL               2024-02-02   
1              INITIAL               2024-08-08   
2         RECTIFICATIF               2024-08-23   
3             

## **3. Modèles de Machine Learning**


### **3.1 Modèle K-Nearest Neighbors Optimisé**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

import pandas as pd

# Charger le fichier .pkl
data_preprocessed = pd.read_pickle("data_preprocessed.pkl")


# Définir la variable cible
y = data_preprocessed['cal_réponse_signalement']

# Séparer les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Initialiser le modèle de K-NN
knn = KNeighborsClassifier()

# Définir les paramètres à tester
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],   # Nombre de voisins
    'weights': ['uniform', 'distance'],  # Pondération des voisins
    'p': [1, 2]  # Distance de Minkowski : 1 = Manhattan, 2 = Euclidienne
}

# Configurer la recherche par grille
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Exécuter la recherche par grille
grid_search.fit(X_train, y_train)

# Meilleurs paramètres et score
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score de validation:", grid_search.best_score_)

# Entraîner le modèle K-NN optimisé
best_knn = grid_search.best_estimator_
y_pred = best_knn.predict(X_test)

# Évaluer les performances
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy avec K-NN optimisé:", accuracy)
print("\nClassification Report:\n", report)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:1103: UserWarning: One or more of the test scores are non-finite: [    nan 0.6385  0.6385  0.639       nan 0.7275  0.67775 0.67725     nan
 0.64    0.69625 0.69625     nan 0.7275  0.70775 0.70775     nan 0.72725
 0.71025 0.71025]
  warnings.warn(


Meilleurs paramètres: {'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
Meilleur score de validation: 0.7275
Accuracy avec K-NN optimisé: 0.741

Classification Report:
                      precision    recall  f1-score   support

     Pris en compte       0.74      1.00      0.85       741
Rejete (hors specs)       0.00      0.00      0.00       259

           accuracy                           0.74      1000
          macro avg       0.37      0.50      0.43      1000
       weighted avg       0.55      0.74      0.63      1000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Dans un premier temps, nous avons testé le modèle des K plus proches voisins (KNN) en optimisant plusieurs paramètres : le nombre de voisins (3, 5, 7), le type de distance (Manhattan ou Euclidienne) et la pondération des voisins (uniforme ou en fonction de la distance). Après une recherche par grille avec validation croisée, la meilleure configuration obtenue est 5 voisins, la distance de Manhattan et une pondération par distance.
Le modèle atteint une accuracy de 74 %, principalement grâce à sa capacité à bien prédire les annonces "Pris en compte", qui dominent les données. En revanche, il échoue presque totalement sur la classe "Rejetés", avec des scores de précision et de rappel proches de zéro. Cela montre que le modèle favorise systématiquement la classe majoritaire, ce qui limite sa performance dans un contexte de données déséquilibrées.
Même après optimisation, le KNN ne semble pas adapté pour ce problème. Il serait donc intéressant d'explorer des modèles plus robustes comme la Régression Logistique ou la Forêt Aléatoire, qui gèrent mieux les déséquilibres de classes.

### **3.2 Modèle LogisticRegression Optimisé**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Assurons-nous que 'X_tfidf' est bien défini
# Exemple (décommenter si nécessaire) :
# tfidf_vectorizer = TfidfVectorizer()
# X_tfidf = tfidf_vectorizer.fit_transform(data_preprocessed['cleaned_text'])

# Définir la variable cible
y = data_preprocessed['cal_réponse_signalement']

# Séparer les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Initialiser le modèle de régression logistique
log_reg = LogisticRegression(max_iter=1000)

# Définir les paramètres à tester
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],               # Paramètre de régularisation
    'penalty': ['l1', 'l2', 'elasticnet'],      # Type de régularisation
    'solver': ['saga'],                         # Saga fonctionne avec l1, l2 et elasticnet
    'l1_ratio': [0, 0.5, 1]                     # Ratio pour elasticnet, 0 pour l2, 1 pour l1
}

# Configurer la recherche par grille
grid_search = GridSearchCV(log_reg, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Exécuter la recherche par grille
grid_search.fit(X_train, y_train)

# Meilleurs paramètres et score
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score de validation:", grid_search.best_score_)

# Entraîner le modèle de régression logistique optimisé
best_log_reg = grid_search.best_estimator_
y_pred = best_log_reg.predict(X_test)

# Évaluer les performances
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy avec Régression Logistique optimisée:", accuracy)
print("\nClassification Report:\n", report)



Nous passons à l'application du modèle de régression logistique. Une fois le code optimisé, on obtient les paramètres {'C': 10, 'l1_ratio': 0, 'penalty': 'l2', 'solver': 'saga'}. Nous sommes conscients qu'avec une régularisation à 10 (ce qui est plutôt grand), nous ajustons un peu mieux nos données (risque de surapprentissage) et les autres paramètres assurent également que le modèle est capable s'adapter à n'importe quelles données sans grands changements. Contrairement au modèle knn, les "rejeté" sont bien mieux détectés.



In [ ]:
# Initialiser le modèle de régression logistique avec les meilleurs paramètres
best_log_reg = LogisticRegression(C=10, l1_ratio=0, penalty='l2', solver='saga', max_iter=1000)

# Entraîner le modèle avec les données d'entraînement
best_log_reg.fit(X_train, y_train)

# Faire des prédictions sur les données de test
y_pred = best_log_reg.predict(X_test)

# Évaluer les performances du modèle
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy avec Régression Logistique optimisée:", accuracy)
print("\nClassification Report:\n", report)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1197: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


Accuracy avec Régression Logistique optimisée: 0.77

Classification Report:
                      precision    recall  f1-score   support

     Pris en compte       0.77      0.98      0.86       741
Rejete (hors specs)       0.74      0.17      0.28       259

           accuracy                           0.77      1000
          macro avg       0.75      0.58      0.57      1000
       weighted avg       0.76      0.77      0.71      1000



Ainsi, nous pouvons voir que le modèle de régression logistique affiche un accuracy score de 0.77.

### **3.3 Modèle DecisionTreeClassifier Optimisé**

In [ ]:
# cal_réponse_signalement est notre variable cible
y = data_preprocessed['cal_réponse_signalement']

# Séparer les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Initialiser le modèle d'arbre de décision
tree = DecisionTreeClassifier(random_state=42)

# Définir les paramètres à tester
param_grid = {
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Configurer et exécuter la recherche par grille
grid_search = GridSearchCV(tree, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Meilleurs paramètres et score
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score de validation:", grid_search.best_score_)

# Entraîner le modèle optimisé avec les meilleurs paramètres
best_tree = grid_search.best_estimator_
y_pred = best_tree.predict(X_test)

# Évaluer les performances
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy avec Arbre de Décision optimisé:", accuracy)
print("\nClassification Report:\n", report)

Meilleurs paramètres: {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 10}
Meilleur score de validation: 0.7220000000000001
Accuracy avec Arbre de Décision optimisé: 0.76

Classification Report:
                      precision    recall  f1-score   support

     Pris en compte       0.77      0.96      0.86       741
Rejete (hors specs)       0.61      0.20      0.30       259

           accuracy                           0.76      1000
          macro avg       0.69      0.58      0.58      1000
       weighted avg       0.73      0.76      0.71      1000



Dans ce code, l'objectif était d'optimiser les paramètres d'un modèle DecisionTreeClassifier en utilisant GridSearchCV. On commence par diviser les données en deux ensembles : 80% pour l’entraînement et 20% pour le test, ce qui permet d’évaluer correctement les performances du modèle sur des données qu’il n’a jamais vues. Ensuite, on définit une grille de paramètres pour tester différentes configurations : la profondeur maximale de l’arbre (max_depth), le nombre minimal d'échantillons pour diviser un nœud (min_samples_split) et le nombre minimal d'échantillons par feuille (min_samples_leaf). GridSearchCV s’occupe d’évaluer toutes les combinaisons possibles grâce à une validation croisée pour trouver les meilleurs réglages. À la fin de l’optimisation, les meilleurs paramètres obtenus sont max_depth=5, min_samples_leaf=4 et min_samples_split=10.

Le modèle est ensuite testé sur l’ensemble de test avec une accuracy de 0.76, ce qui est plutôt satisfaisant. On remarque que la classe "Pris en compte" est très bien prédite avec un recall de 96%, tandis que la classe "Rejeté" pose plus de difficultés avec un recall de seulement 20%. Cela indique que le modèle est efficace pour détecter la classe majoritaire, mais il a du mal à reconnaître correctement les échantillons appartenant à la classe minoritaire.

In [ ]:
# prompt: Meilleurs paramètres: {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 10}

import pandas as pd
import nltk
import re
import spacy
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier


# Initialiser le modèle d'arbre de décision avec les meilleurs paramètres
best_tree = DecisionTreeClassifier(max_depth=5, min_samples_leaf=4, min_samples_split=10, random_state=42)

# Entraîner le modèle avec les données d'entraînement
best_tree.fit(X_train, y_train)

# Faire des prédictions sur les données de test
y_pred = best_tree.predict(X_test)

# Évaluer les performances du modèle
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy avec Arbre de Décision optimisé (avec meilleurs paramètres):", accuracy)
print("\nClassification Report:\n", report)

Accuracy avec Arbre de Décision optimisé (avec meilleurs paramètres): 0.749

Classification Report:
                      precision    recall  f1-score   support

     Pris en compte       0.77      0.94      0.85       741
Rejete (hors specs)       0.54      0.21      0.30       259

           accuracy                           0.75      1000
          macro avg       0.66      0.57      0.58      1000
       weighted avg       0.71      0.75      0.71      1000



Les résultats montrent une accuracy de 0.749, ce qui est très proche du score précédent. Comme dans le code précèdent, on observe que la classe "Pris en compte" est très bien prédite avec un recall de 94%, tandis que la classe "Rejeté" reste difficile à détecter avec seulement 21% de recall. Malgré l'optimisation des paramètres, ce déséquilibre persistant entre les classes montre que le modèle a tendance à favoriser la classe majoritaire. Cela peut s'expliquer par un déséquilibre dans les données où les échantillons "Pris en compte" sont beaucoup plus nombreux.

### **3.4 Modèle RandomForestClassifier Optimisé**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd


# Définir la variable cible
y = data_preprocessed['cal_réponse_signalement']

# Séparer les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Initialiser et entraîner la forêt aléatoire
forest = RandomForestClassifier(n_estimators=100, random_state=42)  # n_estimators est le nombre d'arbres
forest.fit(X_train, y_train)

# Prédire les labels sur l'ensemble de test
y_pred = forest.predict(X_test)

# Évaluer les performances
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy avec Forêt Aléatoire:", accuracy)
print("\nClassification Report:\n", report)

Accuracy avec Forêt Aléatoire: 0.743

Classification Report:
                      precision    recall  f1-score   support

     Pris en compte       0.75      0.99      0.85       741
Rejete (hors specs)       0.57      0.03      0.06       259

           accuracy                           0.74      1000
          macro avg       0.66      0.51      0.45      1000
       weighted avg       0.70      0.74      0.65      1000



Dans ce premier code, on utilise une forêt aléatoire avec des paramètres par défaut. Le modèle s'entraîne sur 80% des données et teste sur les 20% restantes. Les résultats montrent une accuracy de 74,3%, ce qui est correct. Le modèle arrive très bien à détecter les "Pris en compte" avec un rappel presque parfait (99%), mais il galère complètement pour les "Rejeté" où il n'identifie presque rien. Ce problème vient sûrement d'un déséquilibre entre les deux classes.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Charger le fichier .pkl
data_preprocessed = pd.read_pickle("data_preprocessed.pkl")


# Définir la variable cible
y = data_preprocessed['cal_réponse_signalement']

# Séparer les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Initialiser le modèle de forêt aléatoire
forest = RandomForestClassifier(random_state=42)

# Définir les paramètres à tester pour l'optimisation
param_grid = {
    'n_estimators': [50, 100, 200],           # Nombre d'arbres dans la forêt
    'max_depth': [10, 20, None],              # Profondeur maximale de chaque arbre
    'min_samples_split': [2, 5, 10],          # Nombre minimum d'échantillons pour diviser un nœud
    'min_samples_leaf': [1, 2, 4]             # Nombre minimum d'échantillons par feuille
}

# Configurer et exécuter la recherche par grille
grid_search = GridSearchCV(forest, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Meilleurs paramètres et score
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score de validation:", grid_search.best_score_)

# Entraîner le modèle optimisé avec les meilleurs paramètres
best_forest = grid_search.best_estimator_
y_pred = best_forest.predict(X_test)

# Évaluer les performances
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy avec Forêt Aléatoire optimisée:", accuracy)
print("\nClassification Report:\n", report)

Meilleurs paramètres: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
Meilleur score de validation: 0.728
Accuracy avec Forêt Aléatoire optimisée: 0.745

Classification Report:
                      precision    recall  f1-score   support

     Pris en compte       0.75      0.99      0.85       741
Rejete (hors specs)       0.62      0.04      0.07       259

           accuracy                           0.74      1000
          macro avg       0.69      0.52      0.46      1000
       weighted avg       0.72      0.74      0.65      1000



Dans ce deuxième code, on optimise la forêt aléatoire grâce à GridSearchCV, ce qui permet de tester plusieurs paramètres comme le nombre d'arbres, la profondeur et les critères de division. À la fin, on obtient une forêt plus efficace avec les meilleurs paramètres trouvés. L'accuracy monte à 74,5%, donc il y a une petite amélioration. Encore une fois, le modèle détecte très bien les "Pris en compte", mais il a toujours du mal avec les "Rejeté", même si on gagne légèrement en précision. L'optimisation aide un peu, mais pour vraiment améliorer les résultats sur la classe "Rejeté", il faudrait tester d'autres approches.

In [ ]:
# prompt: Meilleurs paramètres: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}

# Initialiser le modèle de forêt aléatoire avec les meilleurs paramètres
best_forest = RandomForestClassifier(max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50, random_state=42)

# Entraîner le modèle avec les données d'entraînement
best_forest.fit(X_train, y_train)

# Faire des prédictions sur les données de test
y_pred = best_forest.predict(X_test)

# Évaluer les performances du modèle
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy avec Forêt Aléatoire optimisée (avec meilleurs paramètres):", accuracy)
print("\nClassification Report:\n", report)

Accuracy avec Forêt Aléatoire optimisée (avec meilleurs paramètres): 0.748

Classification Report:
                      precision    recall  f1-score   support

     Pris en compte       0.75      1.00      0.85       741
Rejete (hors specs)       0.82      0.03      0.07       259

           accuracy                           0.75      1000
          macro avg       0.78      0.52      0.46      1000
       weighted avg       0.77      0.75      0.65      1000



### **3.5 Modèle SVM Optimisé**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split

# --- Étape 1 : Charger les données d'entraînement ---
data_preprocessed = pd.read_pickle("data_preprocessed.pkl")
y_train = data_preprocessed['cal_réponse_signalement']

# --- Étape 2 : Vectoriser les textes d'entraînement ---
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=5, max_df=0.8)
X_train = tfidf_vectorizer.fit_transform(data_preprocessed['cleaned_text'])

# --- Étape 3 : Définir et optimiser le modèle SVM ---
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.01, 0.1],
    'kernel': ['linear', 'rbf']
}

svm = SVC(probability=True, random_state=42)
grid_search = GridSearchCV(svm, param_grid, scoring='f1_weighted', cv=5)

# Recherche des meilleurs paramètres
grid_search.fit(X_train, y_train)
print("Meilleurs paramètres :", grid_search.best_params_)
print("Meilleur score de validation :", grid_search.best_score_)


On commence par charger les données prétraitées et on utilise TF-IDF pour transformer les textes en vecteurs numériques. Ensuite, on définit une grille d’hyperparamètres (C, gamma, et kernel) pour tester différentes configurations du modèle SVM. On utilise GridSearchCV pour trouver les meilleurs paramètres en maximisant le score F1 pondéré sur 5 validations croisées. À la fin, on affiche les meilleurs paramètres et le score obtenu.

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# --- Étape 1 : Charger les données d'entraînement ---
data_preprocessed = pd.read_pickle("data_preprocessed.pkl")
y_train = data_preprocessed['cal_réponse_signalement']

# --- Étape 2 : Vectoriser les textes d'entraînement ---
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=5, max_df=0.8)
X_train = tfidf_vectorizer.fit_transform(data_preprocessed['cleaned_text'])

# --- Étape 3 : Entraîner le modèle SVM avec les meilleurs paramètres ---
best_svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
best_svm.fit(X_train, y_train)

# --- Étape 4 : Évaluation sur les données d'entraînement ---
y_train_pred = best_svm.predict(X_train)
print("Classification Report sur l'ensemble d'entraînement :")
print(classification_report(y_train, y_train_pred))


Après avoir trouvé les meilleurs paramètres, on initialise un modèle SVM avec ces valeurs optimisées. On entraîne ce modèle sur les données prétraitées d’entraînement transformées avec TF-IDF. Ensuite, on réalise des prédictions sur les mêmes données d’entraînement pour vérifier les performances du modèle, et on affiche un rapport de classification pour analyser les résultats.

## **4. Exportation des prédictions**


In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Charger les données prétraitées d'entraînement
data_preprocessed = pd.read_pickle("/content/data_preprocessed.pkl")

# Définir la variable cible
y_train = data_preprocessed['cal_réponse_signalement']

# Créer la matrice TF-IDF pour les données d'entraînement
tfidf_vectorizer = TfidfVectorizer()
X_train = tfidf_vectorizer.fit_transform(data_preprocessed['cleaned_text'])

# Initialiser et entraîner le modèle de régression logistique avec les paramètres optimisés
log_reg = LogisticRegression(C=10, penalty='l2', solver='saga', max_iter=1000)
log_reg.fit(X_train, y_train)

# Charger et prétraiter les données de test
test_data = pd.read_json("/content/test.jsonl", lines=True)
test_data['cleaned_text'] = test_data['texte_annonce'].str.lower().str.replace(r'[^a-z\s]', '', regex=True)

# Transformer les données de test en utilisant le même vectoriseur TF-IDF
X_test = tfidf_vectorizer.transform(test_data['cleaned_text'])

# Effectuer les prédictions sur les données de test
y_pred = log_reg.predict(X_test)

# Ajouter les prédictions au DataFrame de test pour référence
test_data['predictions'] = y_pred

# Exporter les prédictions pour les données de test dans un fichier CSV
test_data[['texte_annonce', 'predictions']].to_csv("test_predictions.csv", index=False)

print("Les prédictions ont été enregistrées dans le fichier 'test_predictions.csv'.")


Les prédictions ont été enregistrées dans le fichier 'test_predictions.csv'.


Ce code entraîne un modèle de régression logistique optimisé avec TF-IDF sur les données d'entraînement. Les données de test sont nettoyées, vectorisées avec le même TF-IDF, puis utilisées pour faire des prédictions. Les résultats sont ajoutés au DataFrame et exportés dans un fichier CSV prêt pour soumission.

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

# Charger les données d'entraînement prétraitées
data_preprocessed = pd.read_pickle("data_preprocessed.pkl")
y_train = data_preprocessed['cal_réponse_signalement']

# Vectoriser les données d'entraînement
tfidf_vectorizer = TfidfVectorizer()
X_train = tfidf_vectorizer.fit_transform(data_preprocessed['cleaned_text'])

# Charger et prétraiter les données de test
test_data = pd.read_json("test.jsonl", lines=True)
test_data['cleaned_text'] = test_data['texte_annonce'].str.lower().str.replace(r'[^a-z\s]', '', regex=True)

# Transformer les données de test avec le même vectoriseur TF-IDF
X_test = tfidf_vectorizer.transform(test_data['cleaned_text'])

# Initialiser et entraîner le modèle de régression logistique
log_reg = LogisticRegression(C=10, penalty='l2', solver='saga', max_iter=1000)
log_reg.fit(X_train, y_train)

# Prédire les valeurs pour cal_réponse_signalement
y_pred = log_reg.predict(X_test)

# Ajouter les prédictions dans le DataFrame de test
test_data['cal_réponse_signalement'] = y_pred

# Créer le fichier final avec uniquement les colonnes OCG_FID et cal_réponse_signalement
final_output = test_data[['OGC_FID', 'cal_réponse_signalement']]

# Exporter le fichier final en CSV
final_output.to_csv("final_test_predictions.csv", index=False)

print("Le fichier 'final_test_predictions.csv' a été généré avec succès.")



Le fichier 'final_test_predictions.csv' a été généré avec succès.


Ce code utilise un modèle de régression logistique optimisé pour prédire la colonne cible à partir des données de test. Après le nettoyage et la vectorisation TF-IDF des textes, les prédictions sont ajoutées au DataFrame avec les colonnes requises, puis exportées en CSV pour une utilisation finale.

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

# Charger les données d'entraînement prétraitées
data_preprocessed = pd.read_pickle("data_preprocessed.pkl")
y_train = data_preprocessed['cal_réponse_signalement']

# Vectoriser les données d'entraînement
tfidf_vectorizer = TfidfVectorizer()
X_train = tfidf_vectorizer.fit_transform(data_preprocessed['cleaned_text'])

# Charger et prétraiter les données de test
test_data = pd.read_json("test.jsonl", lines=True)
test_data['cleaned_text'] = test_data['texte_annonce'].str.lower().str.replace(r'[^a-z\s]', '', regex=True)

# Transformer les données de test avec le même vectoriseur TF-IDF
X_test = tfidf_vectorizer.transform(test_data['cleaned_text'])

# Initialiser le modèle de forêt aléatoire avec les meilleurs paramètres
best_forest = RandomForestClassifier(max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50, random_state=42)

# Entraîner le modèle avec les données d'entraînement
best_forest.fit(X_train, y_train)

# Prédire les valeurs pour cal_réponse_signalement
y_pred = best_forest.predict(X_test)

# Ajouter les prédictions dans le DataFrame de test
test_data['cal_réponse_signalement'] = y_pred

# Créer le fichier final avec uniquement les colonnes OGC_FID et cal_réponse_signalement
final_output = test_data[['OGC_FID', 'cal_réponse_signalement']]

# Exporter le fichier final en CSV
final_output.to_csv("final_test_predictions_forest.csv", index=False)

print("Le fichier 'final_test_predictions_forest.csv' a été généré avec succès.")


Le fichier 'final_test_predictions_forest.csv' a été généré avec succès.


Ce code utilise un Random Forest optimisé pour prédire les valeurs cibles. Il commence par vectoriser les textes en utilisant TF-IDF, puis entraîne le modèle avec les meilleurs hyperparamètres trouvés. Les prédictions sont ajoutées au DataFrame de test et exportées dans un fichier CSV contenant uniquement les colonnes OGC_FID et cal_réponse_signalement.

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
import re

# --- Étape 1 : Charger les données prétraitées d'entraînement ---
data_preprocessed = pd.read_pickle("data_preprocessed.pkl")
y_train = data_preprocessed['cal_réponse_signalement']

# --- Étape 2 : Vectoriser les textes d'entraînement ---
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=5, max_df=0.8)
X_train = tfidf_vectorizer.fit_transform(data_preprocessed['cleaned_text'])

# --- Étape 3 : Charger les données de test ---
test_data = pd.read_json("test.jsonl", lines=True)

# Nettoyer les textes de test
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)  # Supprimer caractères spéciaux et chiffres
    text = re.sub(r'\s+', ' ', text).strip()
    return text

test_data['cleaned_text'] = test_data['texte_annonce'].apply(clean_text)

# Transformer les textes de test avec le vectoriseur TF-IDF déjà entraîné
X_test = tfidf_vectorizer.transform(test_data['cleaned_text'])

# --- Étape 4 : Entraîner et utiliser le modèle optimisé ---
best_svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
best_svm.fit(X_train, y_train)

# Prédire les résultats pour les données de test
y_test_pred = best_svm.predict(X_test)

# Ajouter les prédictions dans le DataFrame de test
test_data['cal_réponse_signalement'] = y_test_pred

# --- Étape 5 : Exporter les prédictions ---
final_output = test_data[['OGC_FID', 'cal_réponse_signalement']]
final_output.to_csv("final_svm_test_predictions.csv", index=False)
print("Le fichier 'final_svm_test_predictions.csv' a été généré avec succès.")


On reprend les données prétraitées et on applique TF-IDF pour les vectoriser, tout comme pour l’entraînement. Les textes des données de test sont nettoyés et transformés avec le même vectoriseur TF-IDF. On utilise le modèle SVM optimisé pour faire des prédictions sur les données de test. Les prédictions sont ajoutées au DataFrame des données de test, puis exportées dans un fichier CSV final pour une analyse ultérieure.

# Partie Brouillon + Test supplémentaire pour Optimisation sans réel succès

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
import pandas as pd
import numpy as np

# Charger le fichier .pkl
data_preprocessed = pd.read_pickle("data_preprocessed.pkl")

# Définir la variable cible
y = data_preprocessed['cal_réponse_signalement']

# Supprimer les colonnes inutiles (par exemple, identifiants ou colonnes textuelles longues)
columns_to_drop = ['OGC_FID', 'texte_annonce']  # Ajuster selon vos données
X = data_preprocessed.drop(columns=columns_to_drop + ['cal_réponse_signalement'], errors='ignore')

# Identifier les colonnes qualitatives restantes et les encoder si nécessaire
from sklearn.preprocessing import LabelEncoder

non_numeric_columns = X.select_dtypes(include=['object']).columns
label_encoder = LabelEncoder()
for col in non_numeric_columns:
    X[col] = label_encoder.fit_transform(X[col])

# Séparer les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Étape 1 : Réduction des variables avec RandomForestClassifier ---
# Entraîner un modèle de forêt aléatoire pour évaluer l'importance des variables
forest = RandomForestClassifier(random_state=42)
forest.fit(X_train, y_train)

# Importance des variables
importances = forest.feature_importances_
importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances}).sort_values(by='Importance', ascending=False)
print("\nTop 10 variables importantes :\n", importance_df.head(10))

# Réduire les variables en ne gardant que celles au-dessus d'un seuil d'importance
threshold = 0.01
important_features = importance_df[importance_df['Importance'] > threshold]['Feature'].tolist()
X_train_reduced = X_train[important_features]
X_test_reduced = X_test[important_features]

# Vérifier les dimensions après réduction
print(f"\nNombre de colonnes après réduction : {X_train_reduced.shape[1]}")



Top 10 variables importantes :
                             Feature  Importance
11                   boamp_intitule    0.065873
2                 boamp_theme_boamp    0.057482
7                boamp_nom_acheteur    0.054459
22           cal_insee_commune_exec    0.054423
25                     cleaned_text    0.052405
16                   cal_mot_rang_1    0.052240
17                   cal_mot_rang_2    0.051991
6   boamp_num_departement_diffusion    0.051126
26                      text_length    0.050604
23             cal_nom_commune_exec    0.050174

Nombre de colonnes après réduction : 23


In [ ]:
# Réentraîner le modèle avec les données originales et gestion des poids
forest_weighted = RandomForestClassifier(
    max_depth=None,
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=50,
    class_weight='balanced',  # Gestion automatique des poids
    random_state=42
)

# Entraîner le modèle
forest_weighted.fit(X_train_reduced, y_train)

# Faire des prédictions sur les données de test
y_pred_weighted = forest_weighted.predict(X_test_reduced)

# Évaluer les performances
accuracy_weighted = accuracy_score(y_test, y_pred_weighted)
report_weighted = classification_report(y_test, y_pred_weighted)

print("\n--- Résultats avec Forêt Aléatoire et Gestion des Poids ---")
print("Accuracy :", accuracy_weighted)
print("\nClassification Report:\n", report_weighted)




--- Résultats avec Forêt Aléatoire et Gestion des Poids ---
Accuracy : 0.753

Classification Report:
                      precision    recall  f1-score   support

     Pris en compte       0.76      0.98      0.85       741
Rejete (hors specs)       0.63      0.11      0.19       259

           accuracy                           0.75      1000
          macro avg       0.69      0.54      0.52      1000
       weighted avg       0.73      0.75      0.68      1000



In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

# Charger les données d'entraînement prétraitées
data_preprocessed = pd.read_pickle("data_preprocessed.pkl")
y_train = data_preprocessed['cal_réponse_signalement']

# Vectoriser les données d'entraînement
tfidf_vectorizer = TfidfVectorizer()
X_train = tfidf_vectorizer.fit_transform(data_preprocessed['cleaned_text'])

# Charger et prétraiter les données de test
test_data = pd.read_json("test.jsonl", lines=True)
test_data['cleaned_text'] = test_data['texte_annonce'].str.lower().str.replace(r'[^a-z\s]', '', regex=True)

# Transformer les données de test avec le même vectoriseur TF-IDF
X_test = tfidf_vectorizer.transform(test_data['cleaned_text'])

# Initialiser et entraîner le modèle de régression logistique
log_reg = LogisticRegression(C=10, penalty='l2', solver='saga', max_iter=1000)
log_reg.fit(X_train, y_train)

# Prédire les valeurs pour cal_réponse_signalement
y_pred = log_reg.predict(X_test)

# Ajouter les prédictions dans le DataFrame de test
test_data['cal_réponse_signalement'] = y_pred

# Créer le fichier final avec uniquement les colonnes OCG_FID et cal_réponse_signalement
final_output = test_data[['OGC_FID', 'cal_réponse_signalement']]

# Exporter le fichier final en CSV
final_output.to_csv("final_test_predictions.csv", index=False)

print("Le fichier 'final_test_predictions.csv' a été généré avec succès.")



Le fichier 'final_test_predictions.csv' a été généré avec succès.


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Charger les données de test avec la vérité terrain
test_data = pd.read_json("test.jsonl", lines=True)
# Charger les prédictions finales
test_predictions = pd.read_csv("/content/final_test_predictions.csv")
# Associer les prédictions aux vérités terrain en utilisant la colonne `OGC_FID`
merged_data = test_data.merge(test_predictions, on='OGC_FID', suffixes=('_réelle', '_prédite'))

# Vérifier si les colonnes nécessaires existent
if 'cal_réponse_signalement' in merged_data.columns and 'cal_réponse_signalement_prédite' in merged_data.columns:
    # Calculer l'accuracy
    accuracy = accuracy_score(merged_data['cal_réponse_signalement'], merged_data['cal_réponse_signalement_prédite'])

    # Générer le rapport de classification
    report = classification_report(merged_data['cal_réponse_signalement'], merged_data['cal_réponse_signalement_prédite'])

    # Matrice de confusion
    cm = confusion_matrix(merged_data['cal_réponse_signalement'], merged_data['cal_réponse_signalement_prédite'])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Rejeté", "Pris en compte"])

    print(f"Accuracy : {accuracy:.3f}\n")
    print("Rapport de classification :\n", report)

    # Afficher la matrice de confusion
    disp.plot(cmap="Blues")
    plt.title("Matrice de Confusion : Modèle Final")
    plt.show()
else:
    print("Les colonnes nécessaires (prédictions ou vérités terrain) ne sont pas disponibles.")


Les colonnes nécessaires (prédictions ou vérités terrain) ne sont pas disponibles.


In [ ]:
important_features = [
    'boamp_intitule', 'boamp_theme_boamp', 'boamp_nom_acheteur',
    'cal_insee_commune_exec', 'cleaned_text', 'cal_mot_rang_1',
    'cal_mot_rang_2', 'boamp_num_departement_diffusion', 'text_length',
    'cal_nom_commune_exec'
]
# Les colonnes utilisées dans votre DataFrame d'entraînement
top_29_features = X_train.columns.tolist()

# Vérifiez le contenu de la liste
print("Liste des 29 features utilisées dans le modèle :")
print(top_29_features)
print(f"Nombre de features : {len(top_29_features)}")


Liste des 29 features utilisées dans le modèle :
['boamp_id_annonce', 'boamp_parent_annonce', 'boamp_theme_boamp', 'boamp_libelle_annonce', 'boamp_statut_annonce', 'boamp_date_fin_de_marche', 'boamp_num_departement_diffusion', 'boamp_nom_acheteur', 'boamp_siret_acheteur', 'boamp_type_organisme', 'boamp_lieu_exec', 'boamp_intitule', 'cal_fichier_pdf', 'cal_num_signalement', 'cal_theme_signalement', 'cal_statut_workflow', 'cal_mot_rang_1', 'cal_mot_rang_2', 'cal_date_traitement', 'cal_type_localisation', 'cal_num_departement_exec', 'cal_siren_epci', 'cal_insee_commune_exec', 'cal_nom_commune_exec', 'cal_adresse_annonce', 'cleaned_text', 'text_length', 'contains_url', 'contains_numbers']
Nombre de features : 29


In [ ]:
# Étape 1 : Charger les données de test
test_data = pd.read_json("test.jsonl", lines=True)

# Étape 2 : Nettoyage et lemmatisation du texte
test_data['cleaned_text'] = test_data['texte_annonce'].apply(clean_text).apply(lemmatize_text)

# Étape 3 : Vectorisation avec le TF-IDF déjà entraîné
# Important : Utiliser le vectoriseur TF-IDF ajusté sur les données d'entraînement
X_test_tfidf = tfidf_vectorizer.transform(test_data['cleaned_text'])

# Étape 4 : Réduction de dimension (SVD)
# Important : Utiliser le modèle SVD déjà entraîné
X_test_reduced = svd.transform(X_test_tfidf)

# Étape 5 : Création des variables additionnelles
test_data['text_length'] = test_data['cleaned_text'].apply(lambda x: len(x.split()))
test_data['contains_url'] = test_data['texte_annonce'].apply(lambda x: 1 if 'http' in x else 0)
test_data['contains_numbers'] = test_data['texte_annonce'].apply(lambda x: 1 if re.search(r'\d', x) else 0)

# Étape 6 : Sélection des colonnes additionnelles et conversion en array
additional_features = test_data[['text_length', 'contains_url', 'contains_numbers']].values

# Étape 7 : Combinaison des variables réduites et additionnelles
X_test_combined = np.hstack((X_test_reduced, additional_features))

# Étape 8 : Sélection des 29 features importants
# Important : Utiliser les mêmes features que ceux sélectionnés pour l'entraînement
selected_features = top_29_features
X_test_final = test_data[selected_features].values



In [ ]:
# Vérification des dimensions
if X_test_final.shape[1] != X_train.shape[1]:
    raise ValueError(f"Incompatibilité : X_test_final a {X_test_final.shape[1]} features, mais le modèle en attend {X_train.shape[1]}.")


In [ ]:
missing_features = [col for col in important_features if col not in test_data.columns]
if missing_features:
    print(f"Colonnes manquantes dans les données de test : {missing_features}")
else:
    print("Toutes les colonnes importantes sont disponibles.")


Toutes les colonnes importantes sont disponibles.


In [ ]:
# Étape 9 : Faire des prédictions avec le modèle optimisé
y_test_pred = best_forest.predict(X_test_final)
y_test_probs = best_forest.predict_proba(X_test_final)[:, 1]

# Étape 10 : Ajouter les prédictions et scores dans le DataFrame de test
test_data['cal_réponse_signalement_prédite'] = y_test_pred
test_data['confidence'] = y_test_probs

# Étape 11 : Exporter les prédictions pour analyse
test_data[['OGC_FID', 'cal_réponse_signalement_prédite']].to_csv("test_predictions.csv", index=False)

print("Les prédictions ont été exportées dans 'test_predictions.csv'.")


NameError: name 'best_forest' is not defined

In [ ]:
if X_test_final.shape[1] != X_train.shape[1]:
    print(f"Incompatibilité : X_test_final a {X_test_final.shape[1]} features, mais le modèle en attend {X_train.shape[1]}.")


Incompatibilité : X_test_final a 103 features, mais le modèle en attend 29.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming 'train_data' is your training data DataFrame
# ... (your previous code to load and preprocess train_data) ...

# 1. Re-fit TF-IDF Vectorizer on the combined text data of train and test sets
all_text_data = pd.concat([train_data['cleaned_text'], test_data['cleaned_text']])
tfidf_vectorizer = TfidfVectorizer()  # Or use your previous parameters
tfidf_vectorizer.fit(all_text_data)

# 2. Transform train and test data with the re-fitted TF-IDF Vectorizer
X_train_tfidf = tfidf_vectorizer.transform(train_data['cleaned_text'])
X_test_tfidf = tfidf_vectorizer.transform(test_data['cleaned_text'])

# 3. Re-fit SVD on the transformed training data
svd.fit(X_train_tfidf)  # Or use your previous parameters

# 4. Transform both train and test data with the re-fitted SVD
X_train_reduced = svd.transform(X_train_tfidf)
X_test_reduced = svd.transform(X_test_tfidf)

# 5. Continue with creating and combining additional features as before
# ... (your existing code for additional features and combining them) ...

# 6. Retrain your model (best_forest) with the new transformed training data
# ... (your existing code for model training) ...

# 7. Now predict using the updated test data
y_test_pred = best_forest.predict(X_test_final)
# ... (rest of your prediction code) ...

NameError: name 'train_data' is not defined

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


# Étape 9 : Analyse des résultats (si étiquettes disponibles dans test_data)
if 'cal_réponse_signalement' in test_data.columns:
    print("\n--- Analyse des Résultats ---")
    print("Classification Report:")
    print(classification_report(test_data['cal_réponse_signalement'], y_test_pred))

    # Matrice de confusion
    cm = confusion_matrix(test_data['cal_réponse_signalement'], y_test_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Rejeté", "Pris en compte"])
    disp.plot(cmap="Blues")
    plt.title("Matrice de Confusion : Forêt Aléatoire Optimisée")
    plt.show()

    # Afficher l'Accuracy
    accuracy = accuracy_score(test_data['cal_réponse_signalement'], y_test_pred)
    print(f"\nAccuracy sur les données de test : {accuracy:.3f}")

# Étape 10 : Exporter les prédictions pour analyse
test_data[['OGC_FID', 'cal_réponse_signalement_prédite', 'confidence']].to_csv("test_predictions.csv", index=False)
print("\nLes prédictions ont été exportées dans 'test_predictions.csv'.")


In [ ]:
# Faire des prédictions avec le modèle optimisé
y_test_pred = best_forest.predict(X_test_final)
y_test_probs = best_forest.predict_proba(X_test_final)[:, 1]

# Ajouter les prédictions et scores dans le DataFrame de test
test_data['cal_réponse_signalement_prédite'] = y_test_pred
test_data['confidence'] = y_test_probs

# Analyse des résultats
print("\n--- Analyse des Résultats ---")
if 'cal_réponse_signalement' in test_data.columns:  # Si les vraies étiquettes sont disponibles
    print("Classification Report:")
    print(classification_report(test_data['cal_réponse_signalement'], y_test_pred))

    # Afficher la matrice de confusion
    cm = confusion_matrix(test_data['cal_réponse_signalement'], y_test_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Rejeté", "Pris en compte"])
    disp.plot(cmap="Blues")
    plt.title("Matrice de Confusion : Forêt Aléatoire Optimisée")
    plt.show()

    # Afficher l'Accuracy
    accuracy = accuracy_score(test_data['cal_réponse_signalement'], y_test_pred)
    print(f"\nAccuracy sur les données de test : {accuracy:.3f}")

# Exporter les prédictions pour analyse
test_data[['OGC_FID', 'cal_réponse_signalement_prédite', 'confidence']].to_csv("test_predictions.csv", index=False)
print("Les prédictions ont été exportées dans 'test_predictions.csv'.")



ValueError: could not convert string to float: "Travaux de création d'une piste cyclable située rue des Poissonniers/Cantelaude/Moulin à Le Teich"

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd

# Charger les données de test
test_data = pd.read_json("test.jsonl", lines=True)

# Réduction des variables sur les données de test
X_test_reduced = test_data[important_features]  # Utilisez les variables importantes identifiées

# Prédire les classes sur les données de test avec le modèle pondéré
y_test_pred_weighted = forest_weighted.predict(X_test_reduced)

# Ajouter les prédictions dans le DataFrame des données de test
test_data['cal_réponse_signalement_prédite'] = y_test_pred_weighted

# Exporter les prédictions
test_data[['OGC_FID', 'cal_réponse_signalement_prédite']].to_csv("test_predictions_weighted.csv", index=False)

print("\nLes prédictions ont été enregistrées dans 'test_predictions_weighted.csv'.")


KeyError: "['cleaned_text', 'text_length'] not in index"